# Example flow for processing and aggregating stats about committee meeting attendees and protocol parts

See the [DataFlows documentation](https://github.com/datahq/dataflows) for more details regarding the Flow object and processing functions.

Feel free to modify and commit changes which demonstrate additional functionality or relevant data.

## Constants

In [115]:
# Limit processing of protocol parts for development
PROCESS_PARTS_LIMIT = 20

# Enable caching of protocol parts data (not efficient, should only be used for local development with sensible PROCESS_PARTS_LIMIT)
PROCESS_PARTS_CACHE = True

# Filter the meetings to be processed, these kwargs are passed along to DataFlows filter_rows processor for meetings resource
MEETINGS_FILTER_ROWS_KWARGS = {'equals': [{'KnessetNum': 20}]}

# Don'e use local data - loads everything from knesset data remote storage
# When set to False - also enables caching, so you won't download from remote storage on 2nd run.
USE_DATA = False

## Loading lexicons

In [67]:
import os

def read_topic_to_set(topic_name):
    lines = open(os.path.join(dir_name, topic_name + ".txt"), 'r').readlines()
    return set([line.strip().replace("\ufeff", "") for line in lines])

dir_name = "../topics/lexicons"        

files = os.listdir(dir_name)

topics = [file.split('.')[0] for file in files]

lexicons = {}
for topic_name in topics:
    lexicons[topic_name] = read_topic_to_list(topic_name)
    
lexicons

## Load source data

In [116]:
from dataflows import filter_rows, cache
from datapackage_pipelines_knesset.common_flow import load_knesset_data, load_member_names

# Loads a dict containing mapping between knesset member id and the member name
member_names = load_member_names(use_data=USE_DATA)

# define flow steps for loading the source committee meetings data
# the actual loading is done later in the Flow
load_steps = (
    load_knesset_data('people/committees/meeting-attendees/datapackage.json', USE_DATA),
    filter_rows(**MEETINGS_FILTER_ROWS_KWARGS)
)

if not USE_DATA:
    # when loading from URL - enable caching which will skip loading on 2nd run
    load_steps = (cache(*load_steps, cache_path='.cache/people-committee-meeting-attendees-knesset-20'),)

loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/members/mk_individual/datapackage.json
using cache data from .cache/members-mk-individual-names
loading from url: https://storage.googleapis.com/knesset-data-pipelines/data/people/committees/meeting-attendees/datapackage.json


## Inspect the datapackages which will be loaded

Last command's output log should contain urls to datapackage.json files, open them and check the table schema to see the resource metadata and available fields which you can use in the processing functions.

Check the [frictionlessdata docs](https://frictionlessdata.io/docs/) for more details about the datapackage file format.

## Extract topics from lines

In [144]:
from collections import defaultdict
from dataflows import Flow

first = True
running_index = 0

meeting_data_global = None
topics_df = None

stats = defaultdict(int)
member_attended_meetings = defaultdict(int)
rows = []

def initialize_meeting_data_global(meeting_row):
    global meeting_data_global
    #global running_index
    meeting_data_global = pd.DataFrame()
    #meeting_data_global['Index'] = running_index
    meeting_data_global['KnessetNum'] = meeting_row['KnessetNum']
    #meeting_data_global['topics'] = ";".join(list(meeting_row['topics']))
    #meeting_data_global['mks'] = ";".join(list(meeting_row['mks']))
    meeting_data_global['CommitteeSessionID'] = meeting_row['CommitteeSessionID']
    meeting_data_global['Number'] = meeting_row['Number']
    for topic_name in lexicons:
        meeting_data_global[topic_name + "_score"] = 0

    
def word_permutations(word):
    clean_word = word.strip()
    permutations = [clean_word]
    if len(word) > 1 and word.startswith('ה') or word.startswith('ב') or word.startswith('ל'):
        permutations.append(word[1:])
    return permutations


def in_lexicon(word, lexicon):
    for p in word_permutations(word):
        if p in lexicon:
            return True
    return False

            
def lexicon_count(lexicon, words):
    count = 0
    for word in words:
        if in_lexicon(word, lexicon):
            count += 1
    return count
        
    
def process_meeting_protocol_part(row):
    global meeting_data_global
    stats['processed parts'] += 1
    words = row['body'].split() if row['body'] is not None else []
    for topic_name, lexicon in lexicons.items():
        meeting_data_global[topic_name + "_score"] += lexicon_count(lexicon, words)  
            

def process_meeting(row):
    global topics_df
    global meeting_data_global
    global running_index
    
    stats['total meetings'] += 1
    if row['attended_mk_individual_ids']:
        for mk_id in row['attended_mk_individual_ids']:
            member_attended_meetings[mk_id] += 1
    parts_filename = row['parts_parsed_filename']
    if parts_filename:
        initialize_meeting_data_global(row)
        if topics_df is None:
            topics_df = meeting_data_global
            print(topics_df)
        else:
            topics_df.append(meeting_data_global, ignore_index=True)
            print(topics_df)

        if PROCESS_PARTS_LIMIT and stats['processed parts'] < PROCESS_PARTS_LIMIT:
            steps = (load_knesset_data('committees/meeting_protocols_parts/' + parts_filename, USE_DATA),)
            if not USE_DATA and PROCESS_PARTS_CACHE:
                steps = (cache(*steps, cache_path='.cache/committee-meeting-protocol-parts/' + parts_filename),)
            steps += (process_meeting_protocol_part,)
            Flow(*steps).process()

process_steps = (process_meeting,)

In [145]:
from dataflows import Flow, dump_to_path

Flow(*load_steps, *process_steps, dump_to_path('data/committee-meeting-attendees-parts')).process()

(<datapackage.package.Package at 0x7f30418cfc88>,
 {'count_of_rows': 10256,
  'bytes': 30129277,
  'hash': '9bb63d3b4c724c88df1416113d0fb80c',
  'dataset_name': None})

In [139]:
topics_df.head(10)

,Index,KnessetNum,CommitteeSessionID,Number,Diplomacy_score,Ecologics_score,Economics_score,Education_score,Health_score,Security_score


## Aggregate and print stats

In [5]:
from collections import deque
import yaml

top_attended_member_names = [member_names[mk_id] for mk_id, num_attended in
                             deque(sorted(member_attended_meetings.items(), key=lambda kv: kv[1]), maxlen=5)]
print('\n')
print('-- top attended members --')
print(top_attended_member_names)
print('\n')
print('-- stats --')
print(yaml.dump(dict(stats), default_flow_style=False, allow_unicode=True))



-- top attended members --
['איתן ברושי', 'מיכאל לוי', 'דוב חנין', 'משה גפני', 'אורי מקלב']


-- stats --
processed parts: 624
total meetings: 9402



## Get output data

Output data is available in the left sidebar under data directory, you can check the datapackage.json and created csv file to explore the data and schema.